In [2]:
#importing libraries
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D, Bidirectional

In [4]:
data = pd.read_csv(r"D:\machine learning\datasets\imdb-dataset-of-50k-movie-reviews\IMDB Dataset.csv")

In [6]:
data.head(3)

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive


In [5]:
#function to clean text data
def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()   # remove HTML tags
    text = re.sub("[^a-zA-Z]", " ", text)                  # keep only letters
    text = text.lower()                                    # lowercase
    return text

data["review"] = data["review"].apply(clean_text)

In [7]:
#encoding target variable
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

C:\Users\manit\AppData\Local\Temp\ipykernel_22024\1644339826.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [8]:
# 4. Train-Test Split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [9]:
# 5. Tokenization because neural networks work with numbers

MAX_WORDS = 10000   # vocab size
MAX_LEN = 300       # review length

tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(train_data["review"])

In [10]:
# Padding since lstm requires fixed length input
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=MAX_LEN)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=MAX_LEN)

Y_train = train_data["sentiment"].values
Y_test = test_data["sentiment"].values

In [12]:
# 6. Build Model

model = Sequential()
model.add(Embedding(input_dim=MAX_WORDS, output_dim=128, input_length=MAX_LEN)) #embedding for word vectors
model.add(SpatialDropout1D(0.2)) #spatial dropout to prevent overfitting
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2))) #bidirectional lstm layer for context from both directions => better understanding

model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

history = model.fit(
    X_train, Y_train,
    batch_size=64,
    epochs=6,
    validation_split=0.2,
    verbose=1
)

Epoch 1/6
500/500 ━━━━━━━━━━━━━━━━━━━━ 549s 1s/step - accuracy: 0.7777 - loss: 0.4742 - val_accuracy: 0.8547 - val_loss: 0.3582
Epoch 2/6
500/500 ━━━━━━━━━━━━━━━━━━━━ 528s 1s/step - accuracy: 0.8613 - loss: 0.3527 - val_accuracy: 0.8493 - val_loss: 0.3727
Epoch 3/6
500/500 ━━━━━━━━━━━━━━━━━━━━ 530s 1s/step - accuracy: 0.8845 - loss: 0.2946 - val_accuracy: 0.8474 - val_loss: 0.4287
Epoch 4/6
500/500 ━━━━━━━━━━━━━━━━━━━━ 522s 1s/step - accuracy: 0.9010 - loss: 0.2647 - val_accuracy: 0.8509 - val_loss: 0.3867
Epoch 5/6
500/500 ━━━━━━━━━━━━━━━━━━━━ 601s 1s/step - accuracy: 0.9045 - loss: 0.2512 - val_accuracy: 0.8396 - val_loss: 0.3962
Epoch 6/6
500/500 ━━━━━━━━━━━━━━━━━━━━ 632s 1s/step - accuracy: 0.9260 - loss: 0.1986 - val_accuracy: 0.8701 - val_loss: 0.3673


In [13]:
# 9. Evaluate Model

loss, accuracy = model.evaluate(X_test, Y_test, verbose=1)
print(f"\n✅ Test Accuracy: {accuracy*100:.2f}%")


# 10. building a prediction function

def predict_sentiment(review):
    review = clean_text(review)#cleaning the review
    seq = tokenizer.texts_to_sequences([review])#tokenizing the review
    padded = pad_sequences(seq, maxlen=MAX_LEN)#padding the review to make it of same length as training data
    pred = model.predict(padded)[0][0]#predicting the sentiment
    sentiment = "Positive 😀" if pred >= 0.5 else "Negative 😞"#since output is given by sigmoid func
    return sentiment, pred

print("\nExample Predictions:")
print(predict_sentiment("This movie was fantastic, I loved it!"))
print(predict_sentiment("Worst movie ever. Waste of time."))

313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.8694 - loss: 0.3612

✅ Test Accuracy: 86.94%

Example Predictions:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step
('Positive 😀', np.float32(0.9881547))
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
('Negative 😞', np.float32(0.0011047827))


In [15]:
# Save the full model to a file
model.save("imdb_sentiment_model.h5")


In [16]:
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
